In [1]:
# Specific Language Impairment Data Set 
# 
# 0 <- explore.R 
#
# This is an R implementation of my SLI classifier
# Version: 15012017 

In [2]:
# Library Headers ------------------------------------------------
library(caret)
library(doParallel)
library(pROC)
library(doMC)

Loading required package: lattice
Loading required package: ggplot2
Loading required package: foreach
Loading required package: iterators
Loading required package: parallel
Type 'citation("pROC")' for a citation.

Attaching package: ‘pROC’

The following objects are masked from ‘package:stats’:

    cov, smooth, var



In [3]:
# Functions ------------------------------------------------------
registerDoMC(cores = 6)

# Load data ------------------------------------------------------
conti4 <- read.csv("Conti4.csv")
eg <- read.csv("EG.csv")
enni <- read.csv("ENNI.csv")
gillam <- read.csv("Gillam.csv")
all3 <- read.csv("all_data.csv")

# Shuffle data ---------------------------------------------------
all3 <- all3[sample(nrow(all3)),]

# 
X <- split(all3, all3$corpus)
all3 <- subset(all3, select = -c(filename,corpus,sex))
lapply(seq_along(X), function(x) {
    assign(c("Conti4", "ENNI", "Gillam")[x], X[[x]], envir=.GlobalEnv)
    }
)

,Y,filename,sex,age,corpus,child_TNW,child_TNS,examiner_TNW,freq_ttr,r_2_i_verbs,⋯,word_errors,f_k,n_v,n_aux,n_3s_v,det_n_pl,det_pl_n,pro_aux,pro_3s_v,total_error
44,0,fskb25.cha,unknown,166,Conti4,334,26,2,0.366,0.18181818,⋯,1,4.15233533,0,0,0,3,0,0,1,4
111,0,fss40.cha,unknown,187,Conti4,241,23,2,0.378,0.20000000,⋯,3,2.98946780,0,1,0,5,0,0,0,6
66,0,fskb47.cha,unknown,168,Conti4,528,59,6,0.345,0.18181818,⋯,3,-0.07634566,1,0,0,10,0,0,2,13
30,0,fskb11.cha,unknown,167,Conti4,256,26,2,0.453,0.16129032,⋯,1,0.64921875,0,0,0,8,0,0,1,9
104,0,fss33.cha,unknown,191,Conti4,262,24,2,0.357,0.17647059,⋯,1,1.77360687,0,0,0,5,0,0,1,6
99,0,fss28.cha,unknown,189,Conti4,209,23,2,0.393,0.31818182,⋯,0,0.31850634,0,0,0,1,0,0,0,1
7,1,fssli501.cha,unknown,180,Conti4,385,27,21,0.373,0.16326531,⋯,1,4.06981241,0,0,0,10,0,0,0,10
2,1,fssli058.cha,unknown,172,Conti4,368,42,27,0.274,0.05000000,⋯,16,1.87170807,0,4,0,5,0,0,0,9
23,0,fskb04.cha,unknown,157,Conti4,735,69,3,0.255,0.43939394,⋯,0,2.45142266,3,14,5,11,0,0,4,37
95,0,fss23.cha,unknown,184,Conti4,350,37,2,0.337,0.17857143,⋯,2,2.66376062,0,0,0,7,0,0,0,7


In [4]:
# Print summary --------------------------------------------------
str(Conti4)
str(eg)
str(ENNI)
str(Gillam)
str(all3)

'data.frame':	118 obs. of  62 variables:
 $ Y                    : int  0 0 0 0 0 0 1 1 0 0 ...
 $ filename             : Factor w/ 1163 levels "10y28837tx.cha",..: 1070 1137 1092 1056 1130 1125 1151 1146 1049 1121 ...
 $ sex                  : Factor w/ 4 levels "female","male",..: 4 4 4 4 4 4 4 4 4 4 ...
 $ age                  : int  166 187 168 167 191 189 180 172 157 184 ...
 $ corpus               : Factor w/ 3 levels "Conti4","ENNI",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ child_TNW            : int  334 241 528 256 262 209 385 368 735 350 ...
 $ child_TNS            : int  26 23 59 26 24 23 27 42 69 37 ...
 $ examiner_TNW         : int  2 2 6 2 2 2 21 27 3 2 ...
 $ freq_ttr             : num  0.366 0.378 0.345 0.453 0.357 0.393 0.373 0.274 0.255 0.337 ...
 $ r_2_i_verbs          : num  0.182 0.2 0.182 0.161 0.176 ...
 $ mor_words            : int  295 241 510 243 252 211 367 361 719 350 ...
 $ num_pos_tags         : int  29 27 38 34 31 30 34 30 46 37 ...
 $ n_dos                : int  0 

'data.frame':	668 obs. of  62 variables:
 $ Y                    : int  0 0 0 0 1 0 1 0 0 1 ...
 $ filename             : Factor w/ 1163 levels "10y28837tx.cha",..: 791 716 375 452 950 792 818 624 619 967 ...
 $ sex                  : Factor w/ 4 levels "female","male",..: 2 2 2 1 1 2 1 1 1 2 ...
 $ age                  : int  99 99 75 74 108 97 99 94 84 89 ...
 $ corpus               : Factor w/ 3 levels "Conti4","ENNI",..: 3 3 3 3 3 3 3 3 3 3 ...
 $ child_TNW            : int  366 270 194 395 307 201 87 122 320 184 ...
 $ child_TNS            : int  46 26 26 39 45 24 17 19 34 26 ...
 $ examiner_TNW         : int  21 1 15 0 36 12 101 29 20 8 ...
 $ freq_ttr             : num  0.389 0.488 0.582 0.341 0.342 0.484 0.651 0.569 0.389 0.387 ...
 $ r_2_i_verbs          : num  0.821 0.214 0.263 0.196 0.56 ...
 $ mor_words            : int  314 244 158 302 272 161 63 109 298 160 ...
 $ num_pos_tags         : int  46 38 40 38 31 37 28 31 41 33 ...
 $ n_dos                : int  0 0 0 0 0 0 0 0 

In [5]:
# Convert class labels to factors --------------------------------
conti4$X[conti4$X==1] <- "SLI"
conti4$X[conti4$X==0] <- "TD"
enni$X[enni$X==1] <- "SLI"
enni$X[enni$X==0] <- "TD"
gillam$X[gillam$X==1] <- "SLI"
gillam$X[gillam$X==0] <- "TD"
eg$X[eg$X==1] <- "SLI"
eg$X[eg$X==0] <- "TD"
all3$Y[all3$Y==1] <- "SLI"
all3$Y[all3$Y==0] <- "TD"

In [6]:
# Train the models -----------------------------------------------
############################ ALL 3 ##############################
set.seed(98)
fitControl <- trainControl(## Repeating k-fold 10 CV
                           method = "cv",
                           number = 10,
                           classProbs = TRUE,
                           allowParallel = TRUE,
                           summaryFunction = twoClassSummary,
                           search = "random",
                           # Try subsampling for class imbalance
                           sampling = "smote")

# Set up parallel processing
registerDoParallel(6)
getDoParWorkers()

[1] 6

In [7]:
# Print out the proportions for each dataset
print('Conti4')
prop.table(table(conti4$X))
print('ENNI')
prop.table(table(enni$X))
print('Gillam')
prop.table(table(gillam$X))
print('EG')
prop.table(table(eg$X))
print('All')
prop.table(table(all3$Y))

[1] "Conti4"



      SLI        TD 
0.1610169 0.8389831 

[1] "ENNI"



     SLI       TD 
0.204244 0.795756 

[1] "Gillam"



     SLI       TD 
0.255988 0.744012 

[1] "EG"



      SLI        TD 
0.2373206 0.7626794 

[1] "All"



      SLI        TD 
0.2295787 0.7704213 

In [ ]:
# xgBoost ------------------------------------------------------------
parametersGrid <-  expand.grid(eta = c(0.02, 0.03), 
                            colsample_bytree=c(0.5,0.7,0.9),
                            max_depth=c(6,8),
                            nrounds=c(100,200),
                            gamma=1,
                            min_child_weight=c(1,2),
                            subsample = c(0.5,1.0)
                            )

modelxgboost <- train(as.factor(Y) ~ ., 
                  data = all3,
                  method = "xgbTree",
                  trControl = fitControl,
                  tuneGrid=parametersGrid)

Warning message in train.default(x, y, weights = w, ...):
“The metric "Accuracy" was not in the result set. ROC will be used instead.”

In [ ]:
plot(modelxgboost)

In [10]:
modelxgboost

eXtreme Gradient Boosting 

1163 samples
  58 predictors
   2 classes: 'SLI', 'TD' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 1046, 1046, 1048, 1047, 1046, 1046, ... 
Addtional sampling using SMOTE

Resampling results across tuning parameters:

  eta   max_depth  colsample_bytree  min_child_weight  subsample  nrounds
  0.01  3          0.5               1                 0.5         10    
  0.01  3          0.5               1                 0.5         50    
  0.01  3          0.5               1                 0.5        100    
  0.01  3          0.5               1                 1.0         10    
  0.01  3          0.5               1                 1.0         50    
  0.01  3          0.5               1                 1.0        100    
  0.01  3          0.5               2                 0.5         10    
  0.01  3          0.5               2                 0.5         50    
  0.01  3          0.5               2          

In [ ]:
# RBF SVM ------------------------------------------------------------
    # Performs a lot better without PCA
    # sigma 0.015298862 C 3.7431066 ROC 0.857222
    # Without nzv takes a long time 
rbfsvm.fit <- train(as.factor(Y) ~ .,data = all3, 
                method = "svmRadial", 
                trControl = fitControl,
                preProcess = c("nzv", "center", "scale"),
                tuneLength = 8,
                metric = "ROC")
plot(rbfsvm.fit)
rbfsvm.fit

In [ ]:
# Linear SVM ------------------------------------------------------------
    # ROC=.855
    # Takes a long time
    # Lower c values < 0.10 perform best
linsvm.fit <- train(as.factor(Y) ~ .,data = all3, 
                method = "svmLinear", 
                trControl = fitControl,
                preProcess = c("nzv", "center", "scale"),
                tuneLength = 8,
                metric = "ROC")
plot(linsvm.fit)
linsvm.fit

In [ ]:
# Boosted Logistic Regression --------------------------------------------
boost_lg.fit <- train(as.factor(Y) ~ .,data = all3, 
                method = "LogitBoost", 
                trControl = fitControl,
                preProcess = c("nzv", "center", "scale"),
                tuneLength = 8,
                metric = "ROC")
plot(boost_lg.fit)
boost_lg.fit

In [ ]:
# Bagged Logistic Regression --------------------------------------------
bag_lg.fit <- train(as.factor(Y) ~ .,data = all3, 
                method = "logicBag", 
                trControl = fitControl,
                preProcess = c("nzv", "center", "scale"),
                tuneLength = 8,
                metric = "ROC")
plot(bag_lg.fit)
bag_lg.fit

In [ ]:
# Logistic Regression --------------------------------------------
lg.fit <- train(as.factor(Y) ~ .,data = all3, 
                method = "logreg", 
                trControl = fitControl,
                preProcess = c("nzv", "center", "scale"),
                tuneLength = 8,
                metric = "ROC")
plot(lg.fit)
lg.fit

In [ ]:
# Neural Network with Selection -----------------------------------
pcnn.fit <- train(as.factor(Y) ~ .,data = all3, 
                  method = "pcaNNet", 
                  trControl = fitControl,
                  preProcess = c("nzv", "center", "scale"),
                  metric = "ROC",
                  tuneLength = 8,
                  verbose = FALSE)
plot(pcnn.fit)
pcnn.fit

In [ ]:
# NN -----------------------------------------------------
nn.fit <- train(as.factor(Y) ~ .,data = all3, 
                  method = "nnet", 
                  trControl = fitControl,
                  preProcess = c("nzv", "center", "scale"),
                  metric = "ROC",
                  tuneLength = 8,
                  verbose = FALSE)
plot(nn.fit)
nn.fit

In [ ]:
# Deep Boost ------------------------------------------------------
db.fit <- train(as.factor(Y) ~ .,data = all3, 
                method = "deepboost", 
                trControl = fitControl,
                preProcess = c("nzv", "center", "scale"))
plot(db.fit)

In [ ]:
# Random Forest --------------------------------------------------
rf.fit <- train(as.factor(Y) ~ .,data = all3, 
                method = "rf", 
                trControl = fitControl,
                preProcess = c("nzv", "center", "scale", "pca"),
                tuneLength = 8,
                metric = "ROC")
plot(rf.fit)

In [ ]:
# Oblique Random Forest ------------------------------------------
orf.fit <- train(as.factor(Y) ~ .,data = all3, 
                 method = "ORFlog", 
                 trControl = fitControl,
                 preProcess = c("nzv", "center", "scale"),
                 tuneLength = 8,
                 metric = "ROC")
plot(orf.fit)

In [ ]:
# Random Forest with Additional Feature Selection ----------------
rf_fs.fit <- train(as.factor(Y) ~ .,data = all3, 
                 method = "Boruta", 
                 trControl = fitControl,
                 preProcess = c("nzv", "center", "scale"),
                 tuneLength = 8,
                 metric = "ROC")
plot(rf_fs.fit)

In [ ]:
# Naive Bayes ----------------------------------------------------
nb.fit <- train(as.factor(Y) ~ .,data = all3, 
                 method = "nb", 
                 trControl = fitControl,
                 preProcess = c("nzv", "center", "scale"),
                 tuneLength = 8,
                 metric = "ROC")
plot(nb.fit)

In [ ]:
# Tree Augmented Naive Bayes -------------------------------------
tan.fit <- train(as.factor(Y) ~ .,data = all3, 
                 method = "tan", 
                 trControl = fitControl,
                 preProcess = c("nzv", "center", "scale"),
                 tuneLength = 8,
                 metric = "ROC")
plot(tan.fit)

In [ ]:
# Model Averaged Naive Bayes -------------------------------------
manb.fit <- train(as.factor(Y) ~ .,data = all3, 
                 method = "manb", 
                 trControl = fitControl,
                 preProcess = c("nzv", "center", "scale"),
                 tuneLength = 8,
                 metric = "ROC")
plot(manb.fit)

In [ ]:
# Stacked Autoencoder Deep NN-------------------------------
dnn.fit <- train(as.factor(Y) ~ .,data = all3, 
                  method = "dnn", 
                  trControl = fitControl,
                  preProcess = c("nzv", "center", "scale"),
                  metric = "ROC")
plot(dnn.fit)
dnn.fit

In [ ]:
# Compare between models
resamps <- resamples(list(RBF_SVM = rbfsvm.fit,
                          Linear_SVM = linsvm.fit,
                          Boosted_Logistic_Regression = boost_lg.fit,
                          Logistic_Regression = lg.fit,
                          PCA_Neural_Network = pcnn.fit,
                          Neural_Network = nn.fit,
                          Random_Forest = rf.fit,
                          Random_Forest_Selection = rf_fs.fit,
                          Oblique_Random_Forest = orf.fit,
                          Naive_Bayes = nb.fit))
summary(resamps)
trellis.par.set(caretTheme())
bwplot(resamps, layout = c(3, 1))

In [ ]:
dotplot(resamps, metric = "ROC")

In [ ]:
splom(resamps)

In [ ]:
difValues <- diff(resamps)
summary(difValues)

In [ ]:
bwplot(difValues, layout = c(3, 1))

In [ ]:
dotplot(difValues)

In [ ]:
# ROC -------------------------------------------------------------
roc.fit <- train(as.factor(X) ~ .,data = conti4, 
                method = "rocc", 
                trControl = fitControl,
                preProcess = c("nzv", "center", "scale"))

In [ ]:
## Cost sensitive learning ----------------------------------------
#c5.fit <- train(as.factor(X) ~ .,data = conti4, 
#                method = "C5.0Cost", 
#                trControl = fitControl,
#                preProcess = c("nzv", "center", "scale"),
#                metric = "Accuracy",
#                verbose = FALSE)

In [ ]:
## AdaBoost M1 ----------------------------------------------------
#fitControl <- trainControl(## Leave One Out CV
#                           method = "LOOCV",
#                           repeats = 10,
#                           p = 0.7,
#                           classProbs = FALSE,
#                           allowParallel = TRUE)
#
#adaboost.fit <- train(as.factor(X) ~ .,data = conti4, 
#                method = "adaboost", 
#                trControl = fitControl,
#                preProcess = c("nzv", "center", "scale"),
#                metric = "Accuracy")
#

In [ ]:
## Linear Discriminant Analysis -----------------------------------
#lda.fit <- train(as.factor(X) ~ .,data = conti4, 
#                method = "lda", 
#                trControl = fitControl,
#                preProcess = c("nzv", "center", "scale"),
#                metric = "Accuracy")
#

In [ ]:
## Conditional Inference Random Forest ----------------------------
#cforest.fit <- train(as.factor(X) ~ .,data = conti4, 
#                method = "cforest", 
#                trControl = fitControl,
#                preProcess = c("nzv", "center", "scale"),
#                metric = "Accuracy")
#

In [ ]:
## Random Forest with Additional Feature Selection ----------------
#boruta.fit <- train(as.factor(X) ~ .,data = conti4, 
#                method = "Boruta", 
#                trControl = fitControl,
#                preProcess = c("nzv", "center", "scale"),
#                metric = "Accuracy")
#
#

In [ ]:
# Print model results --------------------------------------------
svm.fit
lg.fit
pcnn.fit
db.fit
dnn.fit
roc.fit
#c5.fit
#adaboost.fit
#lda.fit
#orf.fit
#rf.fit
#cforest.fit
#boruta.fit
 
#plot(svm.fit)
#svm.test <- predict(svm.fit, conti4.test, type = "prob")
#conti4.test$SVMprob <- svm.test[,"SLI"]
#conti4.test$SVMclass <- predict(svm.fit, conti4.test)
#confusionMatrix(data = conti4.test$SVMclass, 
#                reference = conti4.test$X, 
#                positive = "SLI")
#svm.ROC <- roc(predictor=conti4.test$SVMprob,
#               response=conti4.test$X)
#svm.ROC$auc
##Area under the curve: 0.8731
#plot(svm.ROC,main="SVM ROC")

In [ ]:
## Extreme Gradient Boosting --------------------------------------
#xgb.fit <- train(as.factor(X) ~ .,data = conti4.train, 
#                method = "xgbTree", 
#                trControl = fitControl,
#                preProcess = c("nzv", "center", "scale"),
#                metric = "ROC")
#xgb.fit
#xgb(xvg.fit)
#xgb.test <- predict(sgb.fit, conti4.test, type = "prob")
#conti4.test$xbgprob <- sgb.test[,"SLI"]
#conti4.test$xbgclass <- predict(sgb.fit, conti4.test)
#confusionMatrix(data = conti4.test$xbgclass, 
#                reference = conti4.test$X, 
#                positive = "SLI")
#xbg.ROC <- roc(predictor=conti4.test$xgbprob,
#               response=conti4.test$X)
#xbg.ROC$auc
##Area under the curve: 0.8731
#plot(xbg.ROC,main="xgb ROC")

In [ ]:
# Stochastic Gradient Boosting -----------------------------------
#fitControl <- trainControl(## Leave One Out CV
#                           method = "LOOCV",
#                           classProbs = FALSE)
#set.seed(46)
#gbm.fit <- train(as.factor(X) ~ ., data = conti4.train, 
#                 method = "gbm", 
#                 trControl = fitControl,
#                 verbose = FALSE,
#                 preProcess = c("center", "scale"))
#

In [ ]:
## Learning Vector Quantization -----------------------------------
#set.seed(46)
#lvq.fit <- train(as.factor(X) ~ ., data = conti4.train, 
#                 method = "lvq", 
#                 trControl = fitControl,
#                 preProcess = c("center", "scale"))
#
# Collect resamples ---------------------------------------------
# Summarize the distributions